<a href="https://colab.research.google.com/github/StankoDiego/SOA_EA_2/blob/main/HPC/Stanko_Diego_ejercicio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introducción

#2. Armado del ambiente

##2.1 Armado del ambiente en CPU

No se requiere la ejecucion previa de algun comando

In [13]:
#@title ##2.1.1 Parámetros de ejecución
#@markdown ---
#@markdown ### Especifique la URL de la imagen:
url_imagen = "https://github.com/StankoDiego/SOA_EA_2/blob/main/HPC/los-paisajes-1-768x512.jpg?raw=true" #@param {type: "string"}

#@markdown ---

# TODO: Mejorar información y resultado de ejecución. 
!wget {url_imagen} -O imagen.jpg

--2020-11-22 11:57:08--  https://github.com/StankoDiego/SOA_EA_2/blob/main/HPC/los-paisajes-1-768x512.jpg?raw=true
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/StankoDiego/SOA_EA_2/raw/main/HPC/los-paisajes-1-768x512.jpg [following]
--2020-11-22 11:57:08--  https://github.com/StankoDiego/SOA_EA_2/raw/main/HPC/los-paisajes-1-768x512.jpg
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/StankoDiego/SOA_EA_2/main/HPC/los-paisajes-1-768x512.jpg [following]
--2020-11-22 11:57:08--  https://raw.githubusercontent.com/StankoDiego/SOA_EA_2/main/HPC/los-paisajes-1-768x512.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

##2.1 Armado del ambiente en GPU

Ir a Entornto de ejecucion -> Cambiar tipo de entorno de ejecucion y seleccionar la opcion GPU

In [45]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 8.3MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
     |████████████████████████████████| 81kB 9.1MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621008 sha256=7bb239b4bbda33904eb604d2cd15ccadf32cdf1d05c349d0d7cb5e8104eea526
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=822094fede8cb7bd34c186b0132d9daa0d28871617ae2014676364f16e1c3c01
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


#3. Desarrollo

##3.1 Desarrollo en CPU

In [ ]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

tamMatriz =  15#@param {type: "number"}

# --------------------------------------------

import numpy
import time

tiempoTotal = time.time() * 1000
matriz_A = numpy.random.randint(5000, size=(tamMatriz, tamMatriz))
matriz_A = matriz_A.astype(numpy.int32())

matriz_B = numpy.random.randint(5000, size=(tamMatriz, tamMatriz))
matriz_B = matriz_B.astype(numpy.int32())

print("---------------------------------------------------")
print("Matriz A:\n",matriz_A)
print("---------------------------------------------------")
print("Matriz B:\n",matriz_B)
print("---------------------------------------------------")

tiempoBucle = time.time() * 1000
matriz_R = numpy.zeros((tamMatriz, tamMatriz))  
matriz_R = matriz_R.astype(numpy.int32())

for i in range (0, tamMatriz):
  for j in range(0, tamMatriz):
    matriz_R[i][j] = matriz_A[i][j] + matriz_B[i][j]

print("Matriz R:\n",matriz_R)
print("---------------------------------------------------")

tiempoTotal = (time.time() * 1000) - tiempoTotal
tiempoBucle = (time.time() * 1000) - tiempoBucle
print("--------------------------------------------------")
print("Tiempo del bucle: " , tiempoBucle , "[ms.]")
print("Tiempo del total: " , tiempoTotal , "[ms.]")

##3.2 Desarrollo en GPU

In [ ]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

tamMatriz =  15#@param {type: "number"}

# --------------------------------------------

import time

import numpy
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

tiempoTotal = (time.time()*1000)

matriz_A_cpu = numpy.random.randint(5000, size=(tamMatriz, tamMatriz))
matriz_A_cpu = matriz_A_cpu.astype(numpy.int32())

matriz_B_cpu = numpy.random.randint(5000, size=(tamMatriz, tamMatriz))
matriz_B_cpu = matriz_B_cpu.astype(numpy.int32())

matriz_R_cpu = numpy.zeros((tamMatriz, tamMatriz)) 
matriz_R_cpu = matriz_R_cpu.astype(numpy.int32()) 

matriz_A_gpu = cuda.mem_alloc(matriz_A_cpu.nbytes)
matriz_B_gpu = cuda.mem_alloc(matriz_B_cpu.nbytes)
matriz_R_gpu = cuda.mem_alloc(matriz_R_cpu.nbytes)

cuda.memcpy_htod(matriz_A_gpu, matriz_A_cpu)
cuda.memcpy_htod(matriz_B_gpu, matriz_B_cpu)
cuda.memcpy_htod(matriz_R_gpu, matriz_R_cpu)

print("---------------------------------------------------")
print("Matriz A:\n",matriz_A_cpu)
print("---------------------------------------------------")
print("Matriz B:\n",matriz_B_cpu)
print("---------------------------------------------------")

module = SourceModule("""
__global__ void matrixMultiplicationKernel(float* A, float* B, float* R,
                                           int N
                                           ) {

    int x = blockIdx.y*blockDim.y+threadIdx.y;
    int y = blockIdx.x*blockDim.x+threadIdx.x;

    if(y < N){
      R[x+ y*N] = A[x+ y*N]  +  B[x+ y*N];
    }
}
""")
kernel = module.get_function("matrixMultiplicationKernel")

dim_hilo_x = 16
dim_bloque_x = numpy.int( (tamMatriz + dim_hilo_x-1) / dim_hilo_x )

dim_hilo_y = 16
dim_bloque_y = numpy.int( (tamMatriz + dim_hilo_y-1) / dim_hilo_y )

print( "Thread: [", dim_hilo_x, ",", dim_hilo_y, " ], Bloque : [", dim_bloque_x, ",", dim_bloque_y, "]" )
print( "Total de Thread: [", dim_hilo_x*dim_bloque_x, ",", dim_hilo_y*dim_bloque_y, " ]", " = ", dim_hilo_x*dim_bloque_x*dim_hilo_y*dim_bloque_y )

tiempo_mult = time.time() * 1000

kernel(matriz_A_gpu, matriz_B_gpu, matriz_R_gpu,
       numpy.int32(tamMatriz),        
       block=( dim_hilo_x, dim_hilo_y, 1), 
       grid=(dim_bloque_x, dim_bloque_y,1)
       )

tiempo_mult = (time.time() *1000) - tiempo_mult

cuda.memcpy_dtoh(matriz_R_cpu, matriz_R_gpu)
tiempoTotal = (time.time() * 1000) - tiempoTotal
print("---------------------------------------------------")
print("Matriz resultado:\n",matriz_R_cpu)
print("---------------------------------------------------")
print("Tiempo TOTAL: ", tiempoTotal, "[ms]" )
print("Tiempo GPU  : ", tiempo_mult, "[ms]" )

#4. Tabla de pasos

##4.1 Tabla de pasos CPU

##4.1 Tabla de pasos GPU

#5. Conclusiones